### Hospital Metadata Creation

Why do we create metadata? Separating the metadata from the parsing steps allows us to improve on each one independently. The metadata from this process allows us to parse directly on the files. 

The goal of the config process is to identify the hospitals that have valid input data and prepare the files for extraction. There are 2 failure modes at this stage: 1) the hospital does not have a valid csv or excel file with all CDM data and 2) the hospital does not have the necessary columns i.e. excludes CPT Code. Inability to parse is not a failure mode at this stage. 

Note: I do not use the classes above, except maybe a HospitalData class. As of now raw dictionaries feel better. I will structure the data in a dictioinary of hospital names mapping to objects, and dump it in a json file. 

We approach the creation of the metadata above in the following steps:
- Folder name extraction: match hospital names to a year and a valid folder where it's data is located.
- Sheet extraction: of the available valid files, find the file and sheet which contains the necessary data. Use keywords and sane defaults to automate as possible, and rely on human input to break ties. 
- Column extraction: on the files obtained on the previous step, find the valid columns index containing the data. Come up with keywords and sane defaults, and rely on human input showing 20 rows of the dataframe). 

In [1]:
# Mental model

class SheetDefinition:
    hasValidSheets: bool # for my purposes, it means that allData is defined and unique
    allData: [str, str] # file, filesheet
    # common25: [str, str]
    # rxData: [str, str]
    
class ColumnDefinition:
    hasValidColumns: bool
    cpt: int
    avgCharge: int
    procedureName: int
    
class HospitalData: 
    hospitalName: str
    year: int
    folderPath: str # actually a path object
    
    sheets: SheetDefinition
    columns: bool

class HospitalsData(dict):
    def addData(name: str, data: HospitalData):
        this['name'] = data
    
# example
sampleData = \
{
    "myHospital": {
        "hospitalName": "myHospital",
        "oshpdId": 123456789,
        "year": 2021,
        "folderPath": "myHospitalPath",
        "hasValidPaths": True,
        "hasValidSheets": True,
        "sheets": {
            "allData": {
                "file": "myFile",
                "sheet": "mySheet"
            }
        },
        "hasValidColumns": True,
        "columns": {
            "cpt": 0,
            "avgCharge": 3,
            "procedureName": 1
        }
    }
}

### Get directories

In [2]:
from pathlib import Path
from pprint import pprint
import re
from tqdm import tqdm
import pandas as pd
import numpy as np
import traceback
import time
import json
import pickle
import warnings
from pandas.core.common import SettingWithCopyWarning
from collections import Counter, defaultdict

warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')
warnings.filterwarnings('ignore', category=SettingWithCopyWarning, module='pandas')

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

YEAR = 2021

In [3]:
metadata = {}

dataPath = Path("../chargemaster-cdm-2021")
processingPath = Path('.') / 'processing'

def extractDirectoryMetadata(dataPath):
    folders = sorted(list(dataPath.iterdir()))

    for folder in folders:
        if not folder.is_dir():
            continue

        metadata[folder.name] = {
            "hospitalName": folder.name,
            "year": YEAR,
            "dirPath": folder
        }
        
extractDirectoryMetadata(dataPath)
pprint(list(metadata.items())[:5])

[('AHMC Anaheim Regional Medical Center',
  {'dirPath': PosixPath('../chargemaster-cdm-2021/AHMC Anaheim Regional Medical Center'),
   'hospitalName': 'AHMC Anaheim Regional Medical Center',
   'year': 2021}),
 ('AHMC Seton Medical Center',
  {'dirPath': PosixPath('../chargemaster-cdm-2021/AHMC Seton Medical Center'),
   'hospitalName': 'AHMC Seton Medical Center',
   'year': 2021}),
 ('Alameda Hospital',
  {'dirPath': PosixPath('../chargemaster-cdm-2021/Alameda Hospital'),
   'hospitalName': 'Alameda Hospital',
   'year': 2021}),
 ('Alhambra Hospital Medical Center',
  {'dirPath': PosixPath('../chargemaster-cdm-2021/Alhambra Hospital Medical Center'),
   'hospitalName': 'Alhambra Hospital Medical Center',
   'year': 2021}),
 ('Alta Bates Summit Medical Center',
  {'dirPath': PosixPath('../chargemaster-cdm-2021/Alta Bates Summit Medical Center'),
   'hospitalName': 'Alta Bates Summit Medical Center',
   'year': 2021})]


### Get file paths for allData

How have I dealt with folders that contain multiple hospital data?
I have manually separated them into separate folders, looking up the name of the unidentified hospitals by looking up facility data using the OSHPD Id. 

How have I dealt with folders that only have invalid data?
Only one instance contained the data dispersed on multiple files without cpt codes. For that, I deleted that instance. In the future I would rather skip it with a comment. 

How have I dealt with undistinguishable non CDM files?
Renamed '106370782_CDM(2)_2021.xlsx' to a name not containing CDM or All. The file contained data about supplies that was not relevant enough. 

In [4]:
### Notes
# lookout for false positives, skip file if rx is present
# skip .db files, skip ~ files
# beware of multiple matching files
# one file per hospital, for now. We will support multiple ids and files later. 
# obtain OSHPD Facility No. from filename Note: some hospitals have multiple facilities => diff numbers. 

In [5]:
def extractFilepathMetadata(metadata):
    #file detection regex
    ALL_DATA_REGEX = re.compile(r"^(\d*).*(all|cdm).*(\.xls[xm]?|csv)$", re.MULTILINE | re.IGNORECASE)
    RX_DATA_REGEX = re.compile(r"^(\d*).*(rx).*(\.xls[xm]?|csv)$", re.MULTILINE | re.IGNORECASE)
    COMMON_25_DATA_REGEX = re.compile(r"^(\d*).*(\D25|common).*(\.xls[xm]?|csv)$", re.MULTILINE | re.IGNORECASE)

    for hospital, hospital_data in tqdm(sorted(list(metadata.items()))):    
        # find good allData files
        filePaths = sorted(list(hospital_data["dirPath"].iterdir()))
        filePaths = [path for path in filePaths if not path.name.startswith('~') and not path.suffix=='.db']
        filePaths = [path for path in filePaths if not RX_DATA_REGEX.match(path.name)]
        filePaths = [path for path in filePaths if ALL_DATA_REGEX.match(path.name)]

        # If there are no files, note to user and continue
        if len(filePaths) == 0:
            note = "This hospital has no candidate files"
            print(note)
            metadata[hospital]["hasValidPaths"]=False
            metadata[hospital]["error"]=note
            continue

        # If there is only 1 possible file, assume it is good and proceed
        elif len(filePaths) == 1:
            # This is the happy scenario. You can manually edit the data until 
            # all files execute this branch, as a short term measure

            hospitalId = ALL_DATA_REGEX.match(filePaths[0].name).groups()[0]
            metadata[hospital]["hasValidPaths"]=True
            metadata[hospital]["oshpdId"]= hospitalId
            metadata[hospital]["filepath"]= filePaths[0]

        else:
            print(f"Problem with {hospital}: Files not found by default")
            print("The candidates are:")
            pprint([path.name for path in filePaths])
            print("Select index (1-indexed) or input 's' to skip with a note")
            # userInput = input()
            userInput = 's'
            if userInput == 's':
                note = "More than 1 file present for 1 facility"
                print(note)
                metadata[hospital]["hasValidPaths"]=False
                metadata[hospital]["error"]=note
                continue

            suggestedIdx = int(userInput)-1
            hospitalId = ALL_DATA_REGEX.match(filePaths[suggestedIdx].name).groups()[0]        
            print(f"Selected hospital {filePaths[suggestedIdx].name} with id {hospitalId}")
            metadata[hospital]["hasValidPaths"]=True
            metadata[hospital]["oshpdId"]= hospitalId
            metadata[hospital]["filepath"]= filePaths[suggestedIdx]

extractFilepathMetadata(metadata)

100%|███████████████████████████████████████| 321/321 [00:00<00:00, 5072.61it/s]

Problem with Joyce Eisenberg Keefer Medical Center: Files not found by default
The candidates are:
['106196404_CDM(1)_2021.xlsx',
 '106196404_CDM(2)_2021.xlsx',
 '106196404_CDM(3)_2021.xlsx',
 '106196404_CDM(4)_2021.xlsx']
Select index (1-indexed) or input 's' to skip with a note
More than 1 file present for 1 facility


In [6]:
failedCount = len([data for data in list(metadata.values())if not data["hasValidPaths"]])
successCount = len([data for data in list(metadata.values())if data["hasValidPaths"]])
print(f"Parsed: {successCount}, failed to parse: {failedCount}")

Parsed: 320, failed to parse: 1


### Get sheet names for allData

**List of manual changes:**
- Select all rows and clear all data format for Community Regional Medical Center - Fresno and Clovis Community Medical Center. This is to prevent an error parsing the excel file. 
- UCSD Medical Center: renamed sheet from CMS rates jan 21 to CMS rates jan 21 (felix rx). This classifies that sheet as rx which helps me parse. 
- Fresno Surgical Hospital: renamed chg master to ChargeMaster
- Kindred Hospital Ontario: delete 2019 chargemaster
- Modoc Medical Center: delete inactive chargemaster
- Childrens Hospital at Orange County and Chiildrens Hospital at Mission: Renamed Revenue Usage sheet to CDM
- Community Regional Medical Center Fresno: rename sheet from 2021 CDM HH to 2021 CDM HH (felix home)
- Dameron Hospital: rename sheet Detail 2020-2021 to Detail 2020-2021 (felix change)
- Keck Hospital: rename sheet Keck Hospital ChgMstr to Keck Hospital ChargeMstr
- USC Norris Cancer Hospital: rename sheet USC Norris Cancer ChgMstr to USC Norris Cancer ChargeMstr

**Not machine readable**
- Good Samaritan Hospital - Bakersfield

**No Valid Columns**
- TBD

In [7]:
def filterNonAllDataSheetNames(sheetNames):
    # Sheet name regex
    COMMON_PROCEDURES_REGEX = re.compile(r"^.*(1045|AB).*$") #don't ignore case!
    COMMON_PROCEDURES_REGEX_2 = re.compile(r"^.*(TOP| 50 |[Cc][Oo][Mm][Mm][oO][nN]).*$", re.MULTILINE | re.IGNORECASE)
    IRRELEVANT_SHEETS = re.compile(r"^.*(%|PCT|CHANGE|PERCENTAGE|REV|REVENUE|DRUGS|SUPPLIES|SUPPLY|INCREASE|REMOVED).*$", re.MULTILINE | re.IGNORECASE)
    IRRELEVANT_SHEETS_2 = re.compile(r"^.*(item.number|proccode|Round|levels|methodology|letter|summary|attestation|narrative|info|notes|board|home|disclosure).*$", re.MULTILINE | re.IGNORECASE)
    RX_SHEETS = re.compile(r"^.*(PHARMACY|RX|Pharmaceutical).*$", re.MULTILINE | re.IGNORECASE)
    NON_FARMACY_CDM = re.compile(r"^.*(Non-Pharmacy CDM).*$", re.MULTILINE | re.IGNORECASE)

    candidates = [sheet for sheet in sheetNames if 
                  not COMMON_PROCEDURES_REGEX.match(sheet) 
                  and not COMMON_PROCEDURES_REGEX_2.match(sheet)
                 and not IRRELEVANT_SHEETS.match(sheet)            
                 and not IRRELEVANT_SHEETS_2.match(sheet)            
                and (not RX_SHEETS.match(sheet) or NON_FARMACY_CDM.match(sheet))
                     ]
    return candidates

def greedyChooseAllDataSheetName(sheetNames):
    # Sheetname Regex
    GREEDY_CDM_1 = re.compile(r"^.*(CDM).*$", re.MULTILINE | re.IGNORECASE)
    GREEDY_CDM_2 = re.compile(r"^.*(master).*$", re.MULTILINE | re.IGNORECASE)

    # if an inpatient and an outpatient file is provided, we will read both. 
    # note the lack of .* since this is a very shallow pattern.
    INPATIENT = re.compile(r"^ip$", re.MULTILINE | re.IGNORECASE)
    OUTPATIENT = re.compile(r"^op$", re.MULTILINE | re.IGNORECASE)

    # Returns the greddy choice and a boolean indicating success in choosing. 
    sheetNames.sort()
    for regex in [GREEDY_CDM_1, GREEDY_CDM_2]:
        for sheet in sheetNames:
            if regex.match(sheet):
                return [sheet], True
    if len(sheetNames) == 2:
        if INPATIENT.match(sheetNames[0]) and OUTPATIENT.match(sheetNames[1]):
            return sheetNames, True
    return sheetNames, False

In [8]:
def saveSheetToCsv(sheet: pd.DataFrame, fileTitle: str, dstPath: Path, safe=False)-> Path:
    idx = 1
    filename = f"{fileTitle}.csv"
    newFilename = filename
    if safe: 
        while dstPath.exists() and newFilename in dstPath.iterdir():
            newFilename = f"{filename}{idx:03}.csv"
    
    resultFilepath = dstPath / newFilename  
    dstPath.mkdir(exist_ok=True, parents=True)
    
    regex = r"(\s+)"
    subst = " "
    headers = [(re.sub(regex, subst, str(header), 0, re.MULTILINE)).strip().lower()
              for header in sheet.columns]
               
    sheet.to_csv(resultFilepath, header=headers)
    return resultFilepath

def addAllDataSheetToHospital(hospital, sheetName, sheet) -> None:
    metadata[hospital]["hasValidSheets"] = True
    if not "sheets" in metadata[hospital]:
        metadata[hospital]["sheets"] = {}
    metadata[hospital]["sheets"][sheetName] = {}
    metadata[hospital]["sheets"][sheetName]['columnMappings'] = \
        {
            "cptCode": None,
            "procedureName": None,
            "avgCharge": None
        }
    sheetCsvFile = saveSheetToCsv(sheet, sheetName, rawSheetCsvPath / hospital)
    metadata[hospital]["sheets"][sheetName]["sheetCsvFile"] = sheetCsvFile

In [9]:
getMetadataFile = lambda x: x['filepath']
rawSheetCsvPath = processingPath / 'raw-csv'
failures = []
allSheetNames = []


for hospital, hospitalData in tqdm(sorted(list(metadata.items()))):
    if not hospitalData["hasValidPaths"]:
        continue
    time.sleep(0.1) #just so that the computer relaxes a bit. 
    filepath = getMetadataFile(hospitalData)
    if filepath.suffix == '.csv':
        addAllDataSheetToHospital(hospital, 'csv', pd.read_csv(filepath, index_col=0))
        continue

    # try:
    excelFile = pd.ExcelFile(filepath)
    sheetNames = excelFile.sheet_names
    sheetNames = filterNonAllDataSheetNames(sheetNames)

    if len(sheetNames) == 0:
        raise Exception("Hospital has no available sheets after filtering")

    elif len(sheetNames) == 1:
        sheetName = sheetNames[0]
        addAllDataSheetToHospital(hospital, sheetName, excelFile.parse(sheetName, index_col=0))

    else:
        sheetNamesChoice, choiceSuccess = greedyChooseAllDataSheetName(sheetNames)
        if choiceSuccess:
            for sheetName in sheetNamesChoice:
                addAllDataSheetToHospital(hospital, sheetName, excelFile.parse(sheetName, index_col=0))
            continue

        print(f"Hospital {hospital} has the following sheetNames:")
        pprint(sheetNames)
        print("Select 1-indexed index to keep that file, press 'a' to keep all, press 's' to skip")
        # userInput = input()
        userInput = 'a' # keep all by default
        if userInput == 'a':
            for sheetName in sheetNames:
                addAllDataSheetToHospital(hospital, sheetName, excelFile.parse(sheetName, index_col=0))
        elif userInput == 's':
            raise Exception(f"Could not select a sheet among {sheetNames}")
        else:
            selectedSheetIdx = int(userInput)-1
            sheetName = sheetNames[selectedSheetIdx]
            addAllDataSheetToHospital(hospital, sheetName, excelFile.parse(sheetName, index_col=0))
                
    # except Exception as e:
    #     # traceback.print_exc()
    #     # print(e)
    #     # print("Failed to read sheet names")
    #     # print("Add a note on why this failed")
    #     # note = input()
    #     print(f"Failed at hospital: {hospital}\nWith error:{e}")
    #     failures.append((hospital, e))

 43%|█████████████████▋                       | 138/321 [02:08<01:54,  1.60it/s]

Hospital Kaweah Delta District Hospital has the following sheetNames:
['Medical Center', 'Rural Health Clinics']
Select 1-indexed index to keep that file, press 'a' to keep all, press 's' to skip


100%|█████████████████████████████████████████| 321/321 [08:34<00:00,  1.60s/it]


In [10]:
counter = 0
for hospital in metadata.keys():
    if "hasValidSheets" in metadata[hospital] and metadata[hospital]["hasValidSheets"]:
        counter+=1
    else:
        print(f"Invalid: {hospital}")
counter, len(metadata.keys()) - counter

Invalid: Joyce Eisenberg Keefer Medical Center


(320, 1)

### Finding the headers

Finding the column mapping for cptCode, avgCharge and procedureName can be approached in multiple steps. 

1) Find the index_col of the file. Create a mapping for the right header in the metadata. Verify that the header contains all relevant fields to reduce work on step 2). 
2) Use the pd.index to transform the header onto a list, then process that list with regex and manual input to map the columns. 

We will catch most of the errors in step 1. 
- Category 1 errors: hospital data is not formatted properly i.e. does not contain CPT column.
- Category 2 errors: we do not expect errors. 

Once we finish this, the data will be ready for parsing. 

In [11]:
#enable scrolling
# from IPython.core.display import display, HTML
# from IPython.display import clear_output
# from collections import defaultdict
# display(HTML("<style>.jp-CodeCell.jp-mod-outputsScrolled .jp-Cell-outputArea { max-height: 50em; max-width: 100em; }</style>"))
    
def getSheetFilepath(hospital, sheet):
    return metadata[hospital]['sheets'][sheet]['sheetCsvFile']
    
def dropMostlyNaColumns(df, definedValueThreshold= .05, inPlace=False):
    if not inPlace:
        df = df.copy()
    nRows = len(df)

    for idx, col in enumerate(df.columns):
        definedValues = sum(~df[col].isna())
        definedRatio = definedValues / nRows
        if definedRatio < definedValueThreshold:
            df.drop(col, axis=1, inplace=True)
    return df

def filepathToDataframe(filepath, sheetName=None, header=0):
    sheet = None
    if filepath.suffix == '.csv':
        sheet = pd.read_csv(filepath, header=header)
    else:
        sheet = pd.read_excel(filepath, sheet_name=sheetName, header=header)
    # sheet.index +=1
    sheet = dropMostlyNaColumns(sheet)
    regex = r"(\s+)"
    subst = " "
    headers = [(re.sub(regex, subst, str(header), 0, re.MULTILINE)).strip().lower()
              for header in sheet.columns]
    sheet.set_axis(headers, axis=1, inplace=True)
    sheet.set_index(sheet.columns[0], inplace=True)
    return sheet


def greedyHeaderChoice(sheet: pd.DataFrame)-> int:
    UNTITLED_REGEX = re.compile(r"^.*(untitled|Unnamed).*$", re.MULTILINE | re.IGNORECASE)
    headers = sheet.columns
    isUntitled = any([UNTITLED_REGEX.match(header) for header in headers])
    if not isUntitled:
        return 0
    else:
        for idx, (_, row) in enumerate(sheet.iterrows()):
            if idx > 50:
                break
            if not any(row.isna()):
                return idx+1
        return None

def getHeaderIdxFromUser(hospital, sheetName, guessHeaderIdx = None, nToDisplay=4) -> int:
    filepath = getSheetFilepath(hospital, sheetName)
    if guessHeaderIdx == None:
        sheet = filepathToDataframe(filepath, sheetName, 0)
        guessHeaderIdx = greedyHeaderChoice(sheet)
        if guessHeaderIdx == None:
            raise Exception("Bad column format")
    # print(f"Guessed index {guessHeaderIdx}")
    sheet = filepathToDataframe(filepath, sheetName, guessHeaderIdx)
    
    if len(sheet.columns) <=0:
        raise Exception("Hospital has 1 or fewer columns")
        return
    
    # display(sheet.head(nToDisplay))
    # userInput = input()
    
#     if userInput == 'q':
#         print("Explain why this is invalid input:")
#         reason = input()
#         raise Exception(reason)
    
#     if userInput == 'n':
#         return getHeaderIdxFromUser(hospital, sheetName, 0, 15)
    
#     if userInput == '':
#         userInput = 0
#     userInput = int(userInput)
    userInput = 0 #use auto
    headerIdx = guessHeaderIdx+userInput
    # print (f"Saving header index for hospital as {headerIdx}")
    return headerIdx, sheet
            
trimmedCsvDir = processingPath / 'trimmed-csv'
def extractColumnHeaders(metadata):
    for idx, (hospital, hospitalData) in tqdm(enumerate(sorted(list(metadata.items())))):
        # clear_output()
        # print(hospital, idx)
        if not hospitalData['hasValidPaths'] or not hospitalData['hasValidSheets']:
            continue
        
        for sheetName, sheetData in hospitalData['sheets'].items():
            metadata[hospital]['sheets'][sheetName]['headerIdx'] = None
            metadata[hospital]['sheets'][sheetName]['hasValidHeaders'] = None
            metadata[hospital]['sheets'][sheetName]['trimmedCsvPath'] = None
            metadata[hospital]['error'] = None
            
            try:
                headerIdx, trimmedSheet = getHeaderIdxFromUser(hospital, sheetName)
                metadata[hospital]['sheets'][sheetName]['headerIdx'] = headerIdx
                metadata[hospital]['sheets'][sheetName]['hasValidHeaders']=True

                trimmedSheetPath = saveSheetToCsv(trimmedSheet, sheetName, trimmedCsvDir / hospital)
                metadata[hospital]['sheets'][sheetName]['trimmedCsvPath'] = trimmedSheetPath
                
            except Exception as e:
                print(e)
                # input()
                metadata[hospital]['error'] = e
                metadata[hospital]['sheets'][sheetName]['hasValidHeaders'] = False
        

                
extractColumnHeaders(metadata)

47it [00:03, 13.87it/s]

Bad column format


86it [00:06, 12.40it/s]

Hospital has 1 or fewer columns


321it [00:26, 12.03it/s]


In [12]:
def savePickleSafe(data, filepath):
    idx = 1
    parentFolder = filepath.parent
    safeFilepath = filepath
    while safeFilepath in list(parentFolder.iterdir()):
        safeFilepath = parentFolder / f'{filepath.name[:-len(filepath.suffix)]}{idx:03}{filepath.suffix}'
        idx+=1
    with open(safeFilepath, 'wb') as handle:
        pickle.dump(data, handle)
    return safeFilepath

def readPickle(filepath):
    data = None
    with open(filepath, 'rb') as handle:
        data = pickle.load(handle)
    return data
    


In [13]:
path = savePickleSafe(metadata, Path('.') / "metadata.pickle")
metadata = readPickle(path)
list(metadata.items())[:1]

[('AHMC Anaheim Regional Medical Center',
  {'hospitalName': 'AHMC Anaheim Regional Medical Center',
   'year': 2021,
   'dirPath': PosixPath('../chargemaster-cdm-2021/AHMC Anaheim Regional Medical Center'),
   'hasValidPaths': True,
   'oshpdId': '106301098',
   'filepath': PosixPath('../chargemaster-cdm-2021/AHMC Anaheim Regional Medical Center/106301098_CDM_All_2021.xlsx'),
   'hasValidSheets': True,
   'sheets': {'Price Transparency CDM': {'columnMappings': {'cptCode': None,
      'procedureName': None,
      'avgCharge': None},
     'sheetCsvFile': PosixPath('processing/raw-csv/AHMC Anaheim Regional Medical Center/Price Transparency CDM.csv'),
     'headerIdx': 2,
     'hasValidHeaders': True,
     'trimmedCsvPath': PosixPath('processing/trimmed-csv/AHMC Anaheim Regional Medical Center/Price Transparency CDM.csv')}},
   'error': None})]

The headers are expected to occupy one row and be complete. 

### Manual Change Notes
The following hospitals had headers spanning multiple rows and were edited to use a single row:
- Ballard Rehab
- Childrens Hospital of Orange
- Heritage Oaks H (also had headers interseded with the data which were removed)
- Madera Community Hospital (also removed an empty column)
- Vibra Rehabilitation Hospital

The following hospitals had an extra, unnamed column that was named or deleted:
- Keck Hospital of USC
- Kindred Hospital - San Francisco Bay Area

The following hospitals had bad data which were unreadable and manually unaddressable:
- Colorado River Medicare: many header rows interseded with the data
- Good Samaritan Hospital: Unreadable, and has only one column. 


### Interesting
St. Joseph Hospital - Eureka has both a CPT Code and an HCPCS column which means that we can use this to relate both, potentially. This may be unlikely because CPT is supposed to be a subset of HCPCS and it is hard that a procedure may involve both codes. 

### Finding the column mappings

The relevant columns are CPT Code, procedure name, and average charge. 

Use the sheet columns, process that list with regex and manual input to map the columns. 

In this step we will catch errors on files that do not contain identifiable column names or that don't have at least 2 relevant column names. 

Relevant procedure fields:
- CPT Code: the standard CPT Code for the medical procedure
- HCPCS Code: the HCPCS Code for the medical procedure
- Avg Charge: the charge reported by the hospital for that procedure
- Procedure description: the description of the procedure.

How do I preserve data integrity? I would like cpt code and hospital id to make a primary key, but the data is messy. Furthermore many hospitals don't have a cpt code and I still want to report their procedures. 

I will process the data as is, including duplicates and missing CPT codes. The final clean csv will contain a standarized dataset of all hospitals. 

In [14]:
structuredCsvDir = Path('.') / 'processing' / 'structured-csv'

IRRELEVANT_REGEX = re.compile(r"^.*(ProcedureID|multiplier|revenue|internal| id|statistical|type|status|comment|note|%|change|previous|#|rev | revenue|department|hsp|eap|ref|date|inpatient|i\/p|ip | ip|pharmacy|category|chg cat|mnemonic|dept|group|revcd|rev_code|fim dp|fim cd|sim cd).*$"
                              , re.MULTILINE | re.IGNORECASE)
IRRELEVANT_REGEX_2 = re.compile(r"^(SJGH Hosp ProFee Price|SJGH CDM|Shrine CDM|cod_dtl_ext_id|cod_dtl_ds|comm|PROC \(CDM\)|total.charges|code|code\.1|HOSPITAL|id|ip|rev|mclcde|s|cdm|cdm.num|ub.code|chg.code|charge.code|chargecode|servicecode|procedure.code|charge.master.code|chrg.code|tx|wgt|t|uos|year|a|FQHC|effective)$"
                                , re.MULTILINE | re.IGNORECASE)

IRRELEVANT_REGEX_3 = re.compile(r"^.*(chg.cod|proc.cd|compute|org|account|item.id|active|payor|lst.chg|m.caid|edit|revc|overrides|mod|pt.chg|rx|cdm.code|increase|percentage|facility|man.price|pct|site|ip/er|inpat|taxable|exception|minimum|fractional|difference|tmmc.cdm|sts|2020).*$"
                                , re.MULTILINE | re.IGNORECASE)
IRRELEVANT_REGEX_4 = re.compile(r"^(.*(service.code|proc.code|charge.*code|charge.number|proc.*code|item.number|px.code|ivnum|code |number|units|charge.*item|sta.cd|int.id).*)$"
                                , re.MULTILINE | re.IGNORECASE)
EXCEPTION_REGEX = re.compile(r"^(PT CHG \$|REVENUE DESC|Charge Code Description|ChargeCode Description)$"
                             , re.MULTILINE | re.IGNORECASE)
PRICE_REGEX = re.compile(r"^(.*(\$|AVG TOTAL CHARGE|price|cur.chg|average|amount|gross|amt|final.charge|rate|cost|fee).*|charge|standard.charge|hospital.charge|unit.charge|charge|.*LOCATION \[IP\/OP\]|CURRENT CHARGE)$"
                         , re.MULTILINE | re.IGNORECASE)
DESC_REGEX = re.compile(r"^(.*(name|desc).*)$"
                        , re.MULTILINE | re.IGNORECASE)
CPT_REGEX = re.compile(r"^(.*(cpt).*)$"
                        , re.MULTILINE | re.IGNORECASE)
HCPC_REGEX = re.compile(r"^(.*(hcpc).*)$"
                        , re.MULTILINE | re.IGNORECASE)

columnNamesMap = defaultdict(list)

for hospital, hospitalData in tqdm(metadata.items()):
    if not hospitalData['hasValidPaths'] or not hospitalData['hasValidSheets']:
            continue
    for sheet, sheetData in metadata[hospital]['sheets'].items():
        if not hospitalData['sheets'][sheet]['hasValidHeaders']:
            continue
        df = pd.read_csv(sheetData['trimmedCsvPath'])
        columnNames = df.columns
        columnMappings = {
            'price': [],
            'desc': [],
            'cpt': [],
            'hcpc': []
        }
        for columnName in columnNames:
            if (IRRELEVANT_REGEX.match(columnName)\
            or IRRELEVANT_REGEX_2.match(columnName)\
            or IRRELEVANT_REGEX_3.match(columnName)\
            or IRRELEVANT_REGEX_4.match(columnName))\
            and not EXCEPTION_REGEX.match(columnName):
                continue
            if PRICE_REGEX.match(columnName):
                columnMappings['price'].append(columnName)
                continue
            if DESC_REGEX.match(columnName):
                columnMappings['desc'].append(columnName)
                continue
            if CPT_REGEX.match(columnName):
                columnMappings['cpt'].append(columnName)
                continue
            if HCPC_REGEX.match(columnName):
                columnMappings['hcpc'].append(columnName)
                continue
                
        outputDf = pd.DataFrame()
        for key, value in columnMappings.items():
            if len(value) <= 0:
                continue
            elif len(value) >1: 
                print(hospital)
                print(columnNames.tolist())
                print(json.dumps(columnMappings, indent=2))
                print()
                userInput = input()
                if userInput == 'e':
                    print('write the error')
                    input()
                continue
            # Only 1 item in the list
            else:
                outputDf[key] = df[value[0]]
        outputDf.set_index('desc', inplace=True, drop=True)
        filepath = saveSheetToCsv(outputDf, sheet, structuredCsvDir / hospital)
        metadata[hospital]['sheets'][sheet]['hasValidStructure']=True
        metadata[hospital]['sheets'][sheet]['structuredCsvPath']=filepath
        metadata[hospital]['sheets'][sheet]['columnMappings']=columnMappings

100%|█████████████████████████████████████████| 321/321 [00:11<00:00, 27.00it/s]


Combined multiple column header to single column:
- BHC Freemont hospital
- BHC Alhambra Hospital
- ST. Agnes Medical Center (single row header had to be moved). 

Deleted columns:
- Childrens Hospital at Mission: OP CHARGES, OP AVG CHARGE, CPT01, CPT02
- Childrens Hospital of Orange County: OP CHARGES, OP AVG CHARGE
- Mission Hospital Regional Medical Center: CMM OUTPATIENT [OP]
- Oak Valley District Hospital: Round
- Providence Holy Cross Medical Center: CHC LOCATION OUTPATIENT [OP]
- Providence Little Co. of Mary Med Ctr - San Pedro: CHC LOCATION OUTPATIENT [OP]
- Providence Little Co. of Mary Med Ctr - Torrance: CHC LOCATION OUTPATIENT [OP]
- Providence ST. Joseph Hospital: CHC LOCATION OUTPATIENT [OP]
- Providence St John's Health Center: CJN LOCATION OUTPATIENT [OP]
- Providence St Joseph Medical Center: CSJ LOCATION OUTPATIENT [OP]
- Providence Tarzana Medical Center: CTZ LOCATION OUTPATIENT [OP]
- ST. Jude Medical Center: CFM OUTPATIENT [OP]
- Mayers Memorial Hospital: Code detail description
- Marin General Hospital: Price 2
- San Joaquin General Hospital: SJGH Hosp ProFee Price
- Kindred Hospital - Westminster: Cost
- Northern Inyo Hospital: default cost, minimum price

Removed empty column on index
- Aurora Behavioral Healthcare - Santa Rosa
- Aurora Charter Oak
- Aurora Las Encinas Hospital
- Aurora San Diego Hospital
- Aurora Vista del Mar Hospital


Column name typo correction:
- Corona Regional Medical Center
- Southwest Healthcare System - Murrieta
- Temecula Valley Hospital

Removed renamed axis in Excel for header:
- Madera Community Hospital

Renamed columns:
- UCSD Medical Center: Default -> price
- Unversity of California Irvine Medical Center: OP -> price
- ST. Rose Hospital: procedure -> procedureId
- Mayers Memorial Hospital: Charge -> price
- College Medical Center: CHARGE_ITEM => CHARGE_DESC
- Marina Del Rey Hospital: Charge_code -> Id
- Regional Medical Center of San Jose: I/PCharge -> Price
- Eastern Plumas Health Care: Procedure Code -> HCPC code

### Parsing the files

The final step before loading is to extract the data from the records of the csv files. We will also apply non-contextual transformations i.e. transformations that are non opinionated before loading this to our data warehouse. 

- If HCPC has value and CPT doesn't, use HCPC value
- Replace CPT code by their match to a 5 digit regex surrounded by 0 or more non digits. Convert to number
- Replace all space characters with spaces on procedures, strip whitespaces
- Convert price to number, multiply by 100, look for negatives and anomalies. 
- Drop NA for desc and price fields
- Report row count per hospital and validate anomalies. 


In [15]:
def fillNaWithColumn(df, consumer, provider):
    mask = df[consumer].isna() | df[consumer].isnull()
    mask = mask & ~ (df[provider].isna() | df[provider].isnull())
    df[consumer].mask(mask, df[provider], inplace=True)
    return df


In [16]:
def cleanColumnWhitespace(column):
    column = column.str.strip('"')
    column = column.str.strip("'")
    column = column.str.strip()
    column = column.str.replace(r'\s+', ' ', regex=True)
    column[column==' '] = pd.NA
    return column

In [17]:
def parseSheet(df):
    # print(f'{hospital}:')
    # print(f"Initial rows: {len(df)}")

    # clean whitespace, clean NAs        
    df['desc'] = cleanColumnWhitespace(df['desc'].astype(str))
    mask = (df['desc']==' ') | (df['desc']=='') | (df['desc'].str.lower() == 'nan') \
        | (df['desc'].str.lower() == 'do not use')
    df['desc'].mask(mask, pd.NA, inplace=True)
    prev = len(df)
    df.dropna(axis=0, subset='desc', inplace=True)
    curr = len(df)
    # print(f"Desc NA: {prev-curr}")
    prev=len(df)
    df.drop_duplicates(subset='desc', inplace=True, keep='last')
    curr=len(df)
    # print(f"Desc Dup: {prev-curr}")

    df['price'] = cleanColumnWhitespace(df['price'].astype(str))
    df['price'].mask((df['price']==' ' )| df['price']=='', pd.NA, inplace=True)
    prev = len(df)
    df.dropna(axis=0, subset='price', inplace=True)
    curr = len(df)
    # print(f"Price NA: {prev-curr}")

    # convert type to num, multiply price by 100
    df['price'] = pd.to_numeric(df['price'], errors='coerce')
    prev=len(df)
    df.dropna(axis=0, subset='price', inplace=True)
    curr = len(df)
    # print(f"Price non-numeric: {prev-curr}")
    df['price'] = (df['price'] * 100).astype(np.int64)

    if 'cpt' in df.columns:
        df['cpt'] = cleanColumnWhitespace(df['cpt'].astype(str))

    if 'hcpc' in df.columns:
        df['hcpc'] = cleanColumnWhitespace(df['hcpc'].astype(str))

    # leverage hcpc data if any
    if 'hcpc' in df.columns:
        if 'cpt' not in df.columns:
            df['cpt'] = df['hcpc']
        else:
            df = fillNaWithColumn(df, 'cpt', 'hcpc')
        df.drop('hcpc', axis=1, inplace=True)

    # parse cpt codes 
    if 'cpt' in df.columns:
        if hospital == 'Los Robles Medical Center':
            df['cpt'] = df['cpt'].str.slice(start=1)

        df['cpt'] = df['cpt'].astype(str)
        df['cpt'] = cleanColumnWhitespace(df['cpt']) 
        mask = ~df['cpt'].str.match(CPT_REGEX)
        df['cpt'].mask(mask, pd.NA, inplace=True)
        df['cpt'] = df['cpt'].str.replace(CPT_REGEX, r"\1", regex=True)

        # Drop duplicates
        mask = df.duplicated(subset='cpt', keep='last')
        mask &= df['cpt'].notnull()
        prev = len(df)
        # display(df)
        df=df.loc[~mask]
        curr = len(df)
        # print(f"CPT Dup: {prev-curr}")
    # print(f"Remaining {len(df)}")
    return df



In [24]:
CPT_REGEX = re.compile(r"^(\d{5})([\- ,]?([^\n]{2})?)*$")
parsedCsvDir = Path('.') / 'processing' / 'parsed-csv'
lowDataHospitals = []

for hospital, hospitalData in tqdm(sorted(metadata.items())):
    if not hospitalData["hasValidPaths"] or not hospitalData["hasValidSheets"]:
        continue
    for sheet, sheetData in hospitalData["sheets"].items():
        if not sheetData["hasValidHeaders"] or not sheetData["hasValidStructure"]:
            continue
            
        df = pd.read_csv(sheetData['structuredCsvPath'])
        df = parseSheet(df)
            
        if len(df) < 100:
            lowDataHospitals.append((hospital, len(df)))
            
        df.set_index('desc', inplace=True)
        resultPath = saveSheetToCsv(df, sheet, parsedCsvDir / hospital)
        metadata[hospital]['sheets'][sheet]['parsedCsvPath'] = resultPath

100%|█████████████████████████████████████████| 321/321 [00:41<00:00,  7.69it/s]


### Hospitals with low data
['Aurora Behavioral Healthcare - Santa Rosa',
 'Aurora Charter Oak',
 'Aurora Las Encinas Hospital',
 'Aurora San Diego Hospital',
 'Aurora Vista del Mar Hospital',
 'BHC Alhambra Hospital',
 'BHC Fremont Hospital',
 'Bakersfield Behavioral Healthcare Hospital',
 'Eastern Plumas Health Care',
 'Heritage Oaks Hospital',
 'Kindred Hospital - Ontario',
 'L.A. Co. - Harbor-UCLA Medical Center',
 'L.A. Co. - Olive View Medical Center',
 'L.A. Co. - Rancho Los Amigos Hospital',
 'L.A. Co. - USC Medical Center',
 'Sierra Vista Hospital']
 
I have obtained low amounts of data from these hospitals (<100 procedures)
- Eastern Plumas Health Care does not report procedure name hence I can't extract data. 
- Kindred Hospital - Ontario contains mostly duplicate names
- Other hospitals did not report many procedures

### Enrich CPT Codes for similar descriptions

In [25]:
descToCptMap = dict()
descCount = Counter()
cptCount = Counter()

totalRecords = 0
cptRecords = 0
inferredCptRecords = 0

for hospitalName, hospitalData in tqdm(metadata.items()):
    if 'sheets' not in hospitalData:
        continue
    for sheetName, sheetData in hospitalData['sheets'].items():
        if 'parsedCsvPath' not in sheetData:
            continue
        df = pd.read_csv(sheetData['parsedCsvPath'], dtype={'desc': str, 'price': int, 'cpt': str})
        df['desc'] = df['desc'].astype(str)
        if 'cpt' not in df.columns:
            continue
        totalRecords += len(df)
        df = df[df['cpt'].notnull()]
        cptRecords += len(df)
        for idx, row in df.iterrows():
            descName = row['desc'].lower()
            descToCptMap[descName] = row['cpt']
            descCount[descName] += 1
            if 'cpt' in df.columns:
                cptCode = int(row['cpt']) if not row['cpt'] == '' else -1
                cptCount[cptCode]+=1

100%|█████████████████████████████████████████| 321/321 [00:12<00:00, 26.52it/s]


In [29]:
for hospitalName, hospitalData in tqdm(metadata.items()):
    if 'sheets' not in hospitalData:
        continue
    for sheetName, sheetData in hospitalData['sheets'].items():
        if 'parsedCsvPath' not in sheetData:
            continue
        df = pd.read_csv(sheetData['parsedCsvPath'], dtype={'desc': str, 'price': int, 'cpt': str})
        df['desc'] = df['desc'].astype(str)
        if 'cpt' not in df.columns:
            df['cpt'] = pd.NA
        mask = df['cpt'].isnull()

        for idx, row in df[mask].iterrows():
            descName = row['desc'].lower()
            if descName in descToCptMap and descToCptMap[descName] != -1:
                inferredCptRecords += 1
                cptCode = descToCptMap[descName]
                df.loc[idx, 'cpt']=cptCode
        
        df['hospital'] = hospitalName
        df.set_index('hospital', inplace=True)   
        df=df[['cpt','desc','price']]
        saveSheetToCsv(df, sheetName, parsedCsvDir / hospitalName)

100%|█████████████████████████████████████████| 321/321 [02:12<00:00,  2.42it/s]


In [30]:
totalRecords, cptRecords, inferredCptRecords

(855039, 135686, 84197)

In [31]:
didReportCptCount = 0
for hospital, hospitalData in metadata.items():
    if not 'sheets' in hospitalData:
        continue
    for sheet, sheetData in hospitalData['sheets'].items():
        if not 'columnMappings' in sheetData or not 'cpt' in sheetData['columnMappings']:
            continue
        if len(sheetData['columnMappings']['cpt']) > 0 or len(sheetData['columnMappings']['hcpc']) > 0:
            didReportCptCount += 1
            continue
didReportCptCount

80

### Some final stats
In total 80 hospitals reported CPT or HCPC codes related to procedures. 
There are 855039 total records reported, where 135686 have a CPT code and 84197 had clear enough names that CPT Code could be inferred from other hospitals.

### Prepare database input

Prepare input with only the rows that have an assigned CPT Code. 

In [36]:
databaseInputDir = Path('processing') / 'database-input'

for hospital, hospitalData in tqdm(metadata.items()):
    if not 'sheets' in hospitalData:
        continue
    for sheet, sheetData in hospitalData['sheets'].items():
        if 'parsedCsvPath' not in sheetData:
            continue
        df = pd.read_csv(sheetData['parsedCsvPath'], 
                         dtype=dict(hospital=str, cpt=str, desc=str, price=int))
        if 'cpt' not in df.columns:
            continue
        df.dropna(axis=0, subset=['cpt'], inplace=True)
        df.set_index('hospital', inplace=True)
        saveSheetToCsv(df, sheet, databaseInputDir / hospital)

100%|█████████████████████████████████████████| 321/321 [00:06<00:00, 47.61it/s]
